# Datos y librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut

path = "data/ARIA/4.PatientDemographics/"
showInfo = 1
p = ut.leer_csv(path + "_Patient.csv", showInfo).drop(columns = ["Citizenship","CreationDate","Language","PatientId2",
                       "MobilePhoneProviderSer","PatientId","FileDataSiteID","PatientUID"])

pa = ut.leer_csv(path + "_PatientAddress.csv", showInfo).drop(columns = ["PrimaryFlag"])

a  = ut.leer_csv(path + "_Address.csv", showInfo).drop(columns = ["County","AddressLine1","AddressLine2","PhoneNumber1",
     'Country',"PhoneNumber2","EMailAddress","OriginationDate","TerminationDate","PostalCode","AddressType","StateOrProvince"])

tr = pd.read_csv("data/ARIA_merged&selected/TreatmentRecord.csv", index_col="TreatmentRecordSer")

Archivo: data/ARIA/4.PatientDemographics/_Patient.csv
Se dejaron 16 columnas.
Se eliminaron 25 columnas:


Archivo: data/ARIA/4.PatientDemographics/_PatientAddress.csv
Se dejaron 3 columnas.
Se eliminaron 5 columnas:


Archivo: data/ARIA/4.PatientDemographics/_Address.csv
Se dejaron 14 columnas.
Se eliminaron 9 columnas:




# Exploración

In [2]:
# ut.resumen_tabla(p)
# ut.info_tabla(p)

# Join

In [3]:
join1 = pa.merge(a, on="AddressSer", how="left", suffixes = ("_pa", "_a")).groupby("PatientSer", as_index=False).first()
join2 = p.merge(join1, on="PatientSer", how='left', suffixes = ("_p", "_paa")).drop(columns="AddressSer")
join2.head()

,FirstName,DateOfBirth,Sex,PatientSer,CityOrTownship
0,Truebeam,NaN,Other,4,None
1,NaN,4/29/2008 12:00:00 AM,Unknown,7,None
2,NaN,4/29/2008 12:00:00 AM,Unknown,8,None
3,NaN,5/19/2008 12:00:00 AM,Unknown,9,None
4,NaN,4/29/2008 12:00:00 AM,Unknown,11,None


In [4]:
join2["DateOfBirth"] = pd.DatetimeIndex( pd.to_datetime(join2["DateOfBirth"])).year
join2.head()

,FirstName,DateOfBirth,Sex,PatientSer,CityOrTownship
0,Truebeam,NaN,Other,4,None
1,NaN,2008.0,Unknown,7,None
2,NaN,2008.0,Unknown,8,None
3,NaN,2008.0,Unknown,9,None
4,NaN,2008.0,Unknown,11,None


# Separar pacientes reales y "de prueba"

In [5]:
l2 = list(pd.read_csv("data/nombres_hombre.csv", skiprows=6, delimiter=";").Nombre.values)
l3 = list(pd.read_csv("data/nombres_mujer.csv" , skiprows=6, delimiter=";").Nombre.values)

import unidecode
l4 = []
for i in l2+l3:
    if isinstance(i, str):
        l4.append(unidecode.unidecode(i))

In [6]:
nombres = join2.FirstName.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')\
            .str.upper().str.split().str[0]

In [7]:
patient_real = join2[ nombres.isin(l4)]
patient_test = join2[~nombres.isin(l4)]
print(len(patient_real))
print(len(patient_test))

6759
165


# Corrección
Reviso aquellos pacientes en 'patient_test' que han sido tratados en las máquinas, por si hubiera quedado alguno como falso negativo.

In [8]:
idx = list(set.intersection(set(tr.PatientSer.unique()), set(patient_test.PatientSer)  ))
aa = patient_test[patient_test.PatientSer.isin(idx)]
aa.shape

(51, 5)

Parece que dentro de este grupo aquellos con 'CityOrTownship' no nulo son reales.

In [9]:
idx_pats_real = list(set.union(set(patient_real.PatientSer), set(aa[aa.CityOrTownship.notnull()].PatientSer)))

In [10]:
patient_real = join2[ join2.PatientSer.isin(idx_pats_real)].drop(columns="FirstName")
patient_test = join2[~join2.PatientSer.isin(idx_pats_real)]

In [11]:
print(patient_real.shape)
print(patient_test.shape)

(6789, 4)
(135, 5)


# Resumen

In [12]:
ut.resumen_tabla(patient_real, 'nans')

6789 registros
4 variables


,nans,nunicos,nunicos_r,dtype
PatientSer,0.00,6789,100.00,int64
Sex,0.01,5,0.07,object
DateOfBirth,0.96,95,1.40,float64
CityOrTownship,2.34,753,11.09,object


# Archivo final

In [13]:
patient_real.to_csv("data/ARIA_merged&selected/Patient_real.csv", index=False)
patient_test.to_csv("data/ARIA_merged&selected/Patient_test.csv", index=False)